# Import Python Package/Library

In [6]:
# Learning on how to prepare PascalVOC to Yolov5 format
# Import Required libaries

import numpy as np # manage on linear algebra
import pandas as pd # data processing, CSV file I/O

from pathlib import Path #Represent file system path

#Manage DOM(The Document Object Model) -> API XML Document
from xml.dom.minidom import parse

from shutil import copyfile #High-Level Operation on Files

import os #Provide function to interact with OS

#Test on os.walk function
# for dirname, _, filenames in os.walk('/home/jarupong/Desktop/helmetDetection'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Preparing For Dataset

In [6]:
#Prepare directory for Yolov5 format
!mkdir -p Dataset/labels
!mkdir -p Dataset/images

In [7]:
#Config on class of object in model
classes = ['helmet','head','person']

# Function for preprocess XML to yolov5 format

In [8]:
#This function will transform PasvalVOC to Yolov5 format
#by parse input and calculate to BB algorithm.
def convertFromXML(parsed_img_size, box):
    x1=int(box[0])
    y1=int(box[1])
    x2=int(box[2])
    y2=int(box[3])

    dw = np.float32(1. / int(parsed_img_size[0]) )
    dh = np.float32(1. / int(parsed_img_size[1]) )

    w = x2 - x1
    h = y2 -y1
    x = x1 + (w/2)
    y = y1 + (h/2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h *dh

    return [x, y, w, h]

In [9]:
#This function will create new annotation file in Yolov5 format
def save_txt_file(img_jpg_file_name, parsed_img_size, label_box):
    save_file_name = '/home/jarupong/Desktop/helmetDetection/Dataset/labels/' + img_jpg_file_name +'.txt'
    print(save_file_name)

    with open(save_file_name, 'a+') as file_path:
        for box in label_box:

            class_num = classes.index(box[0])

            yolov5_box = convertFromXML(parsed_img_size, box[1:])

            file_path.write(f"{class_num} {yolov5_box[0]} {yolov5_box[1]} {yolov5_box[2]} {yolov5_box[3]}\n")

        file_path.flush()
        file_path.close()

In [10]:
#This function will access and parsing on original PasvalVOC format 
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    (print(img_path))

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data

    label_box = []
    for box in objects:
        class_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = box.getElementsByTagName("xmin")[0].childNodes[0].data
        y1 = box.getElementsByTagName("ymin")[0].childNodes[0].data
        x2 = box.getElementsByTagName("xmax")[0].childNodes[0].data
        y2 = box.getElementsByTagName("ymax")[0].childNodes[0].data

        img_jpg_file_name = img_xml_file + '.jpg'
        label_box.append([class_name, x1, y1, x2, y2])

    save_txt_file(img_xml_file, [img_w, img_h], label_box)

In [11]:
!ls Dataset/labels/

hard_hat_workers0.txt	  hard_hat_workers2500.txt  hard_hat_workers4000.txt
hard_hat_workers1000.txt  hard_hat_workers2501.txt  hard_hat_workers4001.txt
hard_hat_workers1001.txt  hard_hat_workers2502.txt  hard_hat_workers4002.txt
hard_hat_workers1002.txt  hard_hat_workers2503.txt  hard_hat_workers4003.txt
hard_hat_workers1003.txt  hard_hat_workers2504.txt  hard_hat_workers4004.txt
hard_hat_workers1004.txt  hard_hat_workers2505.txt  hard_hat_workers4005.txt
hard_hat_workers1005.txt  hard_hat_workers2506.txt  hard_hat_workers4006.txt
hard_hat_workers1006.txt  hard_hat_workers2507.txt  hard_hat_workers4007.txt
hard_hat_workers1007.txt  hard_hat_workers2508.txt  hard_hat_workers4008.txt
hard_hat_workers1008.txt  hard_hat_workers2509.txt  hard_hat_workers4009.txt
hard_hat_workers1009.txt  hard_hat_workers250.txt   hard_hat_workers400.txt
hard_hat_workers100.txt   hard_hat_workers2510.txt  hard_hat_workers4010.txt
hard_hat_workers1010.txt  hard_hat_workers2511.txt  hard_hat_workers4011.txt
ha

In [47]:
files = os.listdir('/home/jarupong/Desktop/helmetDetection/PascalVOC/annotations')
for file in files:
    print("file name:", file)
    file_xml = file.split(".")
    print(file_xml[0]) 
    get_xml_data('/home/jarupong/Desktop/helmetDetection/PascalVOC/annotations', file_xml[0])

file name: hard_hat_workers2656.xml
hard_hat_workers2656
/home/jarupong/Desktop/helmetDetection/PascalVOC/annotations/hard_hat_workers2656.xml
/home/jarupong/Desktop/helmetDetection/Dataset/labels/hard_hat_workers2656.txt
file name: hard_hat_workers1949.xml
hard_hat_workers1949
/home/jarupong/Desktop/helmetDetection/PascalVOC/annotations/hard_hat_workers1949.xml
/home/jarupong/Desktop/helmetDetection/Dataset/labels/hard_hat_workers1949.txt
file name: hard_hat_workers1494.xml
hard_hat_workers1494
/home/jarupong/Desktop/helmetDetection/PascalVOC/annotations/hard_hat_workers1494.xml
/home/jarupong/Desktop/helmetDetection/Dataset/labels/hard_hat_workers1494.txt
file name: hard_hat_workers2224.xml
hard_hat_workers2224
/home/jarupong/Desktop/helmetDetection/PascalVOC/annotations/hard_hat_workers2224.xml
/home/jarupong/Desktop/helmetDetection/Dataset/labels/hard_hat_workers2224.txt
file name: hard_hat_workers3901.xml
hard_hat_workers3901
/home/jarupong/Desktop/helmetDetection/PascalVOC/annota

# Split Dataset to Tran, Test, Validate Set

In [12]:
#Import sklearn to split images in data and keep in list
from sklearn.model_selection import train_test_split
image_list = os.listdir('/home/jarupong/Desktop/helmetDetection/PascalVOC/images')
train_list, test_list = train_test_split(image_list, test_size=0.2, random_state=42)
validate_list, test_list = train_test_split(test_list, test_size=0.5, random_state=42)
print('Total Images: ', len(image_list))
print('Train images: ', len(train_list))
print('Test images:', len(test_list))
print('Validate images', len(validate_list))

Total Images:  5000
Train images:  4000
Test images: 500
Validate images 500


In [37]:
#Transform previous list to split and move real image and labels files
def split_model_image(file_list, img_labels_root, img_source,  mode):

    image_path = Path('/home/jarupong/Desktop/helmetDetection/Dataset/images/'+ mode)
    if not image_path.exists():
        print(f"path {image_path} does not exist")
        os.makedirs(image_path)

    label_path = Path('/home/jarupong/Desktop/helmetDetection/Dataset/labels/'+ mode)
    if not label_path.exists():
        print(f"Path {label_path} does not exist")
        os.makedirs(label_path)

    for file in file_list:
        img_name = file.replace('.png', '')
        img_src_file = img_source+'/'+img_name+'.png'
        label_src_file = img_labels_root+'/'+img_name+'.txt'

        IMG_DICT_DIR = '/home/jarupong/Desktop/helmetDetection/Dataset/images/'+mode
        img_dict_file = IMG_DICT_DIR+'/'+img_name+'.png'
        copyfile(img_src_file, img_dict_file)

        LABEL_DICT_DIR = '/home/jarupong/Desktop/helmetDetection/Dataset/labels/'+mode
        label_dict_file = LABEL_DICT_DIR+'/'+img_name+'.txt'
        copyfile(label_src_file,label_dict_file)  

In [38]:
split_model_image(train_list, '/home/jarupong/Desktop/helmetDetection/Dataset/labels/', '/home/jarupong/Desktop/helmetDetection/PascalVOC/images',"train")

/home/jarupong/Desktop/helmetDetection/Dataset/images/train


In [39]:
split_model_image(test_list, '/home/jarupong/Desktop/helmetDetection/Dataset/labels/', '/home/jarupong/Desktop/helmetDetection/PascalVOC/images',"test")
split_model_image(validate_list, '/home/jarupong/Desktop/helmetDetection/Dataset/labels/', '/home/jarupong/Desktop/helmetDetection/PascalVOC/images',"validate")

/home/jarupong/Desktop/helmetDetection/Dataset/images/test
path /home/jarupong/Desktop/helmetDetection/Dataset/images/test does not exist
Path /home/jarupong/Desktop/helmetDetection/Dataset/labels/test does not exist
/home/jarupong/Desktop/helmetDetection/Dataset/images/validate
path /home/jarupong/Desktop/helmetDetection/Dataset/images/validate does not exist
Path /home/jarupong/Desktop/helmetDetection/Dataset/labels/validate does not exist


In [40]:
!ls /home/jarupong/Desktop/helmetDetection/Dataset/images/

test  train  validate
